# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
#stored results in df_cities
df_cities = pd.read_csv("../output_data/cities.csv")
df_cities


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# Store latitude and longitude in locations
locations = df_cities[["Lat", "Lng"]]

# Stored humidity in a series
humidity = df_cities["Humidity"]

In [42]:

#adding layers to heatmap and setting weights to humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=True, max_intensity= None, point_radius=25)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
#used LOC to filter cities by: A max temperature lower than 80 degrees but higher than 70,wind speed less than 10 mph, Zero cloudiness.

df_cities_preferred = df_cities.loc[((df_cities['Cloudiness'] == 0) & (df_cities['Wind Speed'] < 10) & (df_cities['Max Temp'] > 70) & (df_cities['Max Temp'] < 80)), df_cities.columns]

df_cities_preferred

df_hotels = df_cities_preferred.reset_index()

del df_hotels['index']
df_hotels = pd.DataFrame(df_cities_preferred[["Lat", "Lng"]])



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
### Hotel Map
#* Store into variable named `hotel_df`.
#* Add a "Hotel Name" column to the DataFrame.
#* Set parameters to search for hotels with 5000 meters.
#* Hit the Google Places API for each city's coordinates.
#* Store the first Hotel result into the DataFrame.
#* Plot markers on top of the heatmap.


In [45]:
#count the numer of rows
row_count = df_cities_preferred.shape[0]

hotels_df = df_hotels.reset_index()
hotels_df
hotel_list = []

for i in list(range(row_count)):
    radius = 5000
    Lat = hotels_df['Lat'][i]
    Long = hotels_df['Lng'][i]
    
    time.sleep(0.5)
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters&key={g_key}&location={Lat},{Long}&radius={radius}&type=hotel".format(g_key,Lat,Long,radius)
    response = requests.get(base_url)
    hotels_data = response.json()
    #print(json.dumps(hotels_data, indent=4, sort_keys=True)) 
    hotel_name = hotels_data['results'][0]['name']
    print(hotel_name)
    hotel_list.append(hotel_name)
print(hotel_list)


Opuwo
Capitólio
CIDREIRA
Morondava
Jagalbet
Nador
Mogok
Birjand
['Opuwo', 'Capitólio', 'CIDREIRA', 'Morondava', 'Jagalbet', 'Nador', 'Mogok', 'Birjand']


In [46]:
#putting reindexed DF in separate DF, df_cities_preferred_reindex
df_cities_preferred_reindex = df_cities_preferred.reset_index()
df_cities_preferred_reindex

#Storing the info box stuff into a string and list for the info_box_content
 
info_box_content = []

for i in list(range(len(df_cities_preferred))):
    
    info_line = ""
    
    info_line = "City: \n " + str(df_cities_preferred_reindex['City'][i].capitalize()) + "\nCountry: \n " + str(df_cities_preferred_reindex['Country'][i]) + "\nHotel Name: \n " + str(hotel_list[i]) 
    
    info_box_content.append(info_line)
    #name = name.capitalize()
    
    #info_box_content.append(name)
    
    print(info_line)

info_box_content

City: 
 Opuwo
Country: 
 nan
Hotel Name: 
 Opuwo
City: 
 Ponta do sol
Country: 
 BR
Hotel Name: 
 Capitólio
City: 
 Cidreira
Country: 
 BR
Hotel Name: 
 CIDREIRA
City: 
 Morondava
Country: 
 MG
Hotel Name: 
 Morondava
City: 
 Vaini
Country: 
 IN
Hotel Name: 
 Jagalbet
City: 
 Nador
Country: 
 MA
Hotel Name: 
 Nador
City: 
 Mogok
Country: 
 MM
Hotel Name: 
 Mogok
City: 
 Birjand
Country: 
 IR
Hotel Name: 
 Birjand


['City: \n Opuwo\nCountry: \n nan\nHotel Name: \n Opuwo',
 'City: \n Ponta do sol\nCountry: \n BR\nHotel Name: \n Capitólio',
 'City: \n Cidreira\nCountry: \n BR\nHotel Name: \n CIDREIRA',
 'City: \n Morondava\nCountry: \n MG\nHotel Name: \n Morondava',
 'City: \n Vaini\nCountry: \n IN\nHotel Name: \n Jagalbet',
 'City: \n Nador\nCountry: \n MA\nHotel Name: \n Nador',
 'City: \n Mogok\nCountry: \n MM\nHotel Name: \n Mogok',
 'City: \n Birjand\nCountry: \n IR\nHotel Name: \n Birjand']

In [47]:
fig = gmaps.figure()


df_hotels = df_hotels.reset_index()
del df_hotels['index']
df_hotels
df_hotels = pd.DataFrame(df_cities_preferred[["Lat", "Lng"]])

#add heat layer from previous problem
#heat_layer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=True, max_intensity= None, point_radius=25)

marker_locations = df_cities_preferred[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_locations, info_box_content = [city for city in info_box_content])

fig.add_layer(markers)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [48]:
df_hotels.reset_index()

,index,Lat,Lng
0,7,-18.06,13.84
1,70,-20.63,-46.00
2,88,-30.17,-50.22
3,138,-20.30,44.28
4,139,15.34,74.49
5,176,35.17,-2.93
6,297,22.92,96.51
7,444,32.86,59.22


In [49]:

fig = gmaps.figure()




#add heat layer from previous problem
#heat_layer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=True, max_intensity= None, point_radius=25)

#marker_locations = df_cities_preferred[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_locations, info_box_content=["Poverty Rate: {rate}"])

fig.add_layer(markers)
#fig.add_layer(heat_layer)

fig


IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>
#"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map